In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/caill/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_GA_pasta.json"
JSON_FILE

'Data/results_in_progress_GA_pasta.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)           
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [6]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_GA_pasta.json not found. Saving empty list to new file.
- 0 previous results found.


55

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/55 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [8]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,eMJGDVzTH7mbGNWIfz6jcQ,la-tavola-trattoria-atlanta,La Tavola Trattoria,https://s3-media4.fl.yelpcdn.com/bphoto/EeyMhJ...,False,https://www.yelp.com/biz/la-tavola-trattoria-a...,599,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 33.7824462, 'longitude': -84.3549...",[delivery],$$,"{'address1': '992 Virginia Ave NE', 'address2'...",+14048735430,(404) 873-5430,6270.495216
1,uO4A77jXMPfZmlbf_52zRw,boccalupo-atlanta,BoccaLupo,https://s3-media2.fl.yelpcdn.com/bphoto/txg_k6...,False,https://www.yelp.com/biz/boccalupo-atlanta?adj...,652,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 33.7547495778871, 'longitude': -8...",[delivery],$$,"{'address1': '753 Edgewood Ave NE', 'address2'...",+14045772332,(404) 577-2332,5504.599927
2,TDvnkXWrEj4vBfeAgDDiRQ,postino-buckhead-atlanta-3,Postino Buckhead,https://s3-media1.fl.yelpcdn.com/bphoto/TcnWsJ...,False,https://www.yelp.com/biz/postino-buckhead-atla...,90,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 33.854680161043404, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3655 Roswell Rd NE', 'address2':...",+16786081955,(678) 608-1955,10307.092001
3,kXsvTG6zzQt0xKP8jFUpMQ,mirko-pasta-atlanta,Mirko Pasta,https://s3-media3.fl.yelpcdn.com/bphoto/X1H17e...,False,https://www.yelp.com/biz/mirko-pasta-atlanta?a...,294,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 33.87007, 'longitude': -84.38057}","[pickup, delivery]",$$,"{'address1': '4233 Roswell Rd NE', 'address2':...",+14043032110,(404) 303-2110,11973.167598
4,ERVKNes7rXoQg4J0gWhMdg,forza-storico-atlanta,Forza Storico,https://s3-media3.fl.yelpcdn.com/bphoto/MSNL6R...,False,https://www.yelp.com/biz/forza-storico-atlanta...,403,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 33.78662558409769, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '1198 Howell Mill Rd', 'address2'...",+14044648096,(404) 464-8096,2251.206980


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,eYXjqcBzCgLGu7c2MQPrQg,dominos-pizza-smyrna-4,Domino's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/8rlcxQ...,False,https://www.yelp.com/biz/dominos-pizza-smyrna-...,37,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 33.885624, 'longitude': -84.477874}",[],$,"{'address1': '2754 Cumberland Blvd SE', 'addre...",+17704361313,(770) 436-1313,14163.688989
996,W_qV9cHhAX8EEJoSffN9fQ,pizza-hut-norcross-12,Pizza Hut,https://s3-media3.fl.yelpcdn.com/bphoto/2PH5uI...,False,https://www.yelp.com/biz/pizza-hut-norcross-12...,37,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 33.9697503561217, 'longitude': -8...","[pickup, delivery]",$,"{'address1': '5450 Peachtree Pkwy', 'address2'...",+17702639500,(770) 263-9500,29167.698672
997,HpPr_NDQ4M0jWEYk7SYeUA,pizza-hut-atlanta-16,Pizza Hut,https://s3-media2.fl.yelpcdn.com/bphoto/3NdsUS...,False,https://www.yelp.com/biz/pizza-hut-atlanta-16?...,23,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1.5,"{'latitude': 33.941565, 'longitude': -84.27047...","[pickup, delivery]",$,"{'address1': '4957 Winters Chapel Rd', 'addres...",+17709019800,(770) 901-9800,23786.822271
998,BYQDG27CeZKJAWE_a6D-Cg,hola-taqueria-and-bar-roswell,HOLA! Taqueria & Bar,https://s3-media1.fl.yelpcdn.com/bphoto/jrOsAj...,False,https://www.yelp.com/biz/hola-taqueria-and-bar...,524,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 34.0331289, 'longitude': -84.3404...","[pickup, delivery]",$$,"{'address1': '688 Holcomb Bridge Rd', 'address...",+14702425661,(470) 242-5661,30435.373541
999,tL1bgGbWjpPnRthxJ18Avg,pizza-hut-dallas-16,Pizza Hut,https://s3-media3.fl.yelpcdn.com/bphoto/Fv2VRd...,False,https://www.yelp.com/biz/pizza-hut-dallas-16?a...,6,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.9357690823349, 'longitude': -8...",[delivery],$,"{'address1': '4075 Marietta Hwy', 'address2': ...",+17704438800,(770) 443-8800,33804.495604


In [9]:
# check for duplicate ID's 
#final_df.duplicated(subset='id').sum()

## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [10]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)